# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from scipy.optimize import *
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *
from stlab.devices.RS_SGS100A import RS_SGS100A
from stlab.devices.Keysight_N5183B import Keysight_N5183B

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice import config, RR_1_IF,RO_lo, readout_len, Q1_lo, Q1_IF, sat_Q1_len, gauss_len, R1_RS, Q1_RS, t_Q1_pi, amp_Q1_pi

In [ ]:
from scipy.signal import gaussian
print(readout_len)
gauss_test = 0.5 * gaussian(gauss_len, 0.15* gauss_len)
plt.plot(gauss_test)

# Edit configuration parameters

**To do**

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.184.193::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(-15)
RR.setCWfrequency(RO_lo)
RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
RR.IQon()
RR.write(':SOURce:IQ:IMPairment:STATe ON')

#LO for downconversion
MXG = Keysight_N5183B(addr='TCPIP::192.168.1.91::INSTR',reset=True,verb=True)
MXG.RFon()
MXG.setCWpower(19)
MXG.setCWfrequency(RO_lo)
MXG.INTref()

QDrive = RS_SGS100A("TCPIP::169.254.50.124::INSTR", reset=True,verb=True) 
QDrive.EXTref()
QDrive.RFon()
QDrive.setCWpower(0)
QDrive.setCWfrequency(Q1_lo)
QDrive.write(':SOURce:IQ:IMPairment:LEAKage:I ' + Q1_RS[0])
QDrive.write('SOURce:IQ:IMPairment:LEAKage:Q ' + Q1_RS[1])
QDrive.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + Q1_RS[2])
QDrive.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + Q1_RS[3])
QDrive.IQon()
QDrive.write(':SOURce:IQ:IMPairment:STATe ON')

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# QUA Program

In [ ]:
def qubit_pulse_cal(n_max, f_vec, a_vec, b, QLO_Power):
    QDrive.setCWpower(QLO_Power)
    
    f_min = np.min(f_vec)
    f_max = np.max(f_vec)
    df = f_vec[1]-f_vec[0]
    
    a_min = np.min(a_vec)
    a_max = np.max(a_vec)
    
    if len(a_vec) == 1: 
        da = 1 
    else:
        da = a_vec[1]-a_vec[0]
    
    with program() as qubit_pulse_cal:

        ##############################
        # declare real-time variables:
        ##############################

        n = declare(int)        # Averaging
        f = declare(int)        # Frequencies
        a=declare(fixed)

        A=declare(fixed)
        B=declare(fixed)
        iA=declare(fixed)
        iB=declare(fixed)
        Re=declare(fixed)
        Im=declare(fixed)

        Re_st = declare_stream()
        Im_st = declare_stream()

        ###############
        # the sequence:
        ###############
        with for_(n, 0, n < n_max, n + 1):

            with for_(f, f_min, f < f_max+df/2, f + df):
                
                update_frequency("RR_1", f)

                with for_(a,a_min,a<a_max+da/2,a+da):
                    
                    wait(int(5000/4),"RR_1","Q1")

                    #play("saturation"*amp(a), "Q1")
                    play("gaussian"*amp(a), "Q1", duration = t_Q1_pi)
                    align("Q1", "RR_1")
                    #wait(int((gauss_len-gauss_len/2)/4),"RR_1")

                    measure("readout"*amp(b), "RR_1", None, demod.full("integW_cos", A, "out1"),
                                                                demod.full("integW_sin",B,"out2"),
                                                                demod.full("integW_sin", iA, "out1"),
                                                                demod.full("integW_cos",iB,"out2"))

                    assign(Re, A + B)       
                    assign(Im, iA - iB)
                    save(Re, Re_st)
                    save(Im, Im_st)              

        with stream_processing():
            Re_st.buffer(len(f_vec), len(a_vec)).average().save("Re")
            Im_st.buffer(len(f_vec), len(a_vec)).average().save("Im")
    
    job = qm.execute(qubit_pulse_cal, duration_limit=0, data_limit=0)
    
    res_handles= job.result_handles
    res_handles.wait_for_all_values()

    Re_handle = res_handles.get("Re")
    Im_handle = res_handles.get("Im")
    
    Re_s = Re_handle.fetch_all()
    Im_s = Im_handle.fetch_all()
    
    return Re_s, Im_s

# Define Parameters

In [ ]:
n_max = 5000 #10000

f_min = RR_1_IF-10.0e6 
f_max = RR_1_IF+10e6 
df = 0.5e6 #0.1e6
f_vec = np.arange(f_min, f_max+df/2, df)

#Readout pulse amplitude
b = 0.24

# Qubit pulse amp 
a_min = 0
a_max = 2
#da = 0.005
da = 0.1 #0.025 #0.01
a_vec=np.arange(a_min,a_max+da/2,da)

print(len(a_vec))
print(len(f_vec))
print(a_vec)

In [ ]:
Re, Im = qubit_pulse_cal(n_max, f_vec, a_vec, b, 0)

## Fetch data and save to .dat

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'Q1_Pulse_Cal'

data_Re = Re
data_Im = Im
data_Sig = np.abs(data_Re + 1j*data_Im)
data_Amp = 20*(np.log10(np.abs(data_Sig)))
data_Ph = np.unwrap(np.arctan(data_Im/data_Re))

data = np.asarray([(RO_lo+(f_vec.reshape(len(f_vec),1)+a_vec)-a_vec).T,np.round(((a_vec.reshape(len(a_vec),1))+f_vec)-f_vec,3),data_Re.T,data_Im.T,data_Sig.T,data_Amp.T,data_Ph.T])

for i,qpow in enumerate(a_vec):
    print(i,qpow)
    data_dict={'Frequency (Hz)':data[0][i],
           'Qubit Pulse Amplitude':data[1][i],
           'Real':data[2][i],
           'Imaginary':data[3][i],
           'Signal':data[4][i],
           'Amplitude (dB)':data[5][i],
           'Phase (rad)':data[6][i]
    }
    if i==0:
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True, git_id = False)

        output = new_stdout.getvalue()
        sys.stdout = old_stdout
        print(output)
        M_ind = output.find("Measurement Name")
        M_name = output[M_ind+len('Measurement Name:  '):-1]
    stlab.savedict(myfile,data_dict)
    stlab.metagen.fromarrays(myfile,data_dict['Frequency (Hz)'],a_vec[0:i+1],xtitle='Frequency (Hz)',ytitle='Qubit Pulse Amplitude',colnames=list(data_dict))

# Processing/Plotting

#### Pulse cal frequency sweep

In [ ]:
sig = Re + 1j*Im

power = 20*(np.log10(np.abs(sig)))#+10*np.log10(1000/50)#np.abs(sig)   #equation from wiki
power2=power/np.mean(power,axis=0)                             #norm lbl
power3=(scipy.ndimage.gaussian_filter(1/(power/np.mean(power,axis=0)),[1,5])) #gaussian filtered norm lbl
power4=power-np.mean(power,axis=0)                                    #sub lbl
power5=scipy.ndimage.gaussian_filter(power,[1,3])-np.mean(power,axis=0)#gaussian filtered sub lbl
power6=scipy.ndimage.gaussian_filter(power,[1,3]) #just gaussian filtered
phase = np.unwrap(np.arctan(Im/Re))

plt.figure(num=None, figsize=(8, 6), dpi=100)
plt.tight_layout()
#plt.plot(RO_lo+f_vec,power)
#wait_time = 0 

X, Y = np.meshgrid((RO_lo+f_vec)/1e9, a_vec)
plt.contourf(X,Y, power.T,levels=100,norm=colors.PowerNorm(gamma=2.5),cmap='RdBu')
# #plt.plot((XY2_lo+f_vec)/1e9,power)
plt.xlabel('Frequency (GHz)')
# plt.title('Qubit PowSpec, t_Q={}ns, t_RO={}ns, wait={}ns, R_amp = {}, longw={}ns'.format(sat_Q1_len,readout_len,wait_time,b,5*sat_Q1_len))
# #plt.title('Qubit PowSpec, t_Q={}ns, t_RO={}ns, align, R={}, longw={}'.format(sat_Q2_len,readout_len,b,3*wait_time))
# #plt.xticks(X[0,0],X[0,-1])
plt.ylabel('Qubit Pulse Amplitude')
cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 15
# cbar.ax.set_ylabel('Normalized Signal Amplitude', rotation=270);

# Save this file and configuration file to measurement folder

In [ ]:
#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_BMDevice.py')
save_config_path = os.path.join(meas_path, 'Configuration_BMDevice.py')

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);

#### Some other program

In [ ]:
def qubit_pulse_len_cal(n_max, f_vec, a, b, t_vec, QLO_Power):
    Re_t = np.zeros([len(f_vec),len(t_vec)])
    Im_t = np.zeros([len(f_vec),len(t_vec)])
    
    for i in range(len(t_vec)):
        config['pulses']['gaussian_pulse']['length'] = t_vec[i]
        config['waveforms']['gauss_wf']['samples'] = 0.5 * gaussian(t_vec[i], 0.25*t_vec[i])

        qmm = QuantumMachinesManager()
        qm = qmm.open_qm(config)

        QDrive.setCWpower(QLO_Power)

        f_min = np.min(f_vec)
        f_max = np.max(f_vec)
        df = f_vec[1]-f_vec[0]

        with program() as qubit_pulse_len_cal:

            ##############################
            # declare real-time variables:
            ##############################

            n = declare(int)        # Averaging
            f = declare(int)        # Frequencies

            A=declare(fixed)
            B=declare(fixed)
            iA=declare(fixed)
            iB=declare(fixed)
            Re=declare(fixed)
            Im=declare(fixed)

            Re_st = declare_stream()
            Im_st = declare_stream()

            ###############
            # the sequence:
            ###############
            with for_(n, 0, n < n_max, n + 1):

                with for_(f, f_min, f < f_max+df/2, f + df):

                    update_frequency("RR_1", f)                
                    #with for_(t,t_min,t < t_max+dt/2,t+dt):
                    wait(int(2500/4),"RR_1","Q1")
                    #play("saturation"*amp(a), "Q1")
                    play("gaussian"*amp(a), "Q1", duration = t)
                    align("Q1", "RR_1")
                    #wait(int((sat_Q1_len-readout_len/2)/4),"RR_1")

                    measure("readout"*amp(b), "RR_1", None, demod.full("integW_cos", A, "out1"),
                                                                demod.full("integW_sin",B,"out2"),
                                                                demod.full("integW_sin", iA, "out1"),
                                                                demod.full("integW_cos",iB,"out2"))

                    assign(Re, A + B)       
                    assign(Im, iA - iB)
                    save(Re, Re_st)
                    save(Im, Im_st)              

            with stream_processing():
                Re_st.buffer(len(f_vec)).average().save("Re")
                Im_st.buffer(len(f_vec)).average().save("Im")

        job = qm.execute(qubit_pulse_len_cal, duration_limit=0, data_limit=0)

        res_handles= job.result_handles
        res_handles.wait_for_all_values()

        Re_handle = res_handles.get("Re")
        Im_handle = res_handles.get("Im")

        Re_s = Re_handle.fetch_all()
        Im_s = Im_handle.fetch_all()
        
        Re_t[:,i] = Re_s
        Im_t[:,i] = Im_s
    
    return Re_t, Im_t